# S3W11D2: Agentic RAG - 给 RAG 装上“开关”

## 🎯 今日目标 (Today's Goals)

1. **架构进化**: 从 `User -> RAG` 进化为 `User -> Agent (Router) -> Tools/Chat`.
2. **工程落地**:
    * 完善 `src/llm/tools.py`: 封装 `search_knowledge_base` 工具。
    * 创建 `src/rag/agent.py`: 实现智能路由逻辑。


3. **算法攻坚**: 掌握 BST (二叉搜索树) 的核心验证逻辑 (LC 98)。



## 1. 理论知识：Router 模式 (The "Brain")

在 Agentic RAG 中，LLM 扮演 **Router (路由器)** 的角色。

* **场景 A**: 用户问 "Hello"。
    * LLM 思考: "这不需要外部知识。"
    * Action: 直接回复 "Hello! How can I help you?"


* **场景 B**: 用户问 "Transformer 的 Attention 机制是什么？"
    * LLM 思考: "这是一个专业问题，我需要查阅知识库。"
    * Action: 调用 `search_knowledge_base(query="Transformer Attention mechanism")`。
    * Observation: 获取文本块。
    * Final Response: 基于文本块回答。



**第一性原理**: 我们其实是在利用 LLM 的 **语义理解能力** 来做 **意图分类 (Intent Classification)**，只是这个分类的结果直接转化为了函数调用。



## 2. 工程实现 (Engineering)

我们需要修改/创建两个核心文件。

### 2.1 完善工具箱 (`src/llm/tools.py`)

我们需要把之前的向量检索逻辑封装成一个工具。
*(注意：这里假设你之前的 Week 9 代码能跑通。如果暂时不想连接真实的 ChromaDB，可以用下面的 Mock 数据代替，先把链路跑通)*

```python
# src/llm/tools.py
import json

# 尝试导入你之前写的 RAG 组件
# 如果之前的代码路径不同，请调整 import
try:
    # 假设你在 src.rag.vector_db 里有个 query_vector_db 函数
    # from src.rag.vector_db import query_vector_db 
    pass 
except ImportError:
    pass

def search_knowledge_base(query: str) -> str:
    """
    根据用户的查询，在本地向量知识库(Vector DB)中检索相关文档。
    """
    print(f"🔍 [Tool]: 正在知识库中检索: {query} ...")
    
    # --- 这里是连接真实 RAG 的接口 ---
    # 真实场景：results = query_vector_db(query, top_k=3)
    # 真实场景：return json.dumps(results)
    
    # --- 模拟数据 (Mock) ---
    # 为了今天先跑通 Agent 逻辑，我们先返回模拟数据
    mock_db = {
        "transformer": "Transformer 是一种基于自注意力机制(Self-Attention)的深度学习模型，由 Google 在 2017 年提出。",
        "rag": "RAG (Retrieval-Augmented Generation) 是一种结合了检索和生成的架构，用于解决 LLM 的幻觉问题。",
        "resnet": "ResNet (残差网络) 通过引入 Skip Connection 解决了深层网络难以训练的问题。"
    }
    
    for key, value in mock_db.items():
        if key in query.lower():
            return json.dumps({"status": "success", "content": value})
            
    return json.dumps({"status": "empty", "content": "知识库中未找到相关内容，请尝试换个关键词。"})

# 定义工具 Schema
TOOLS_SCHEMA = [
    {
        "type": "function",
        "function": {
            "name": "search_knowledge_base",
            "description": "当用户询问具体的技术概念、定义或需要查阅内部文档时调用此工具。不要用于日常闲聊。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "用于检索的关键词或问题，例如 'Transformer原理' ",
                    },
                },
                "required": ["query"],
            },
        },
    }
]

# 函数映射表
AVAILABLE_FUNCTIONS = {
    "search_knowledge_base": search_knowledge_base,
}

```


### 2.2 构建 Agent 大脑 (`src/rag/agent.py`)

这是今天的重头戏。我们将创建一个 `Agent` 类来管理对话循环。

```python
# src/rag/agent.py
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
from src.llm.tools import TOOLS_SCHEMA, AVAILABLE_FUNCTIONS

load_dotenv()

class RAGAgent:
    def __init__(self):
        # 初始化 DeepSeek / SiliconFlow 客户端
        self.client = OpenAI(
            api_key=os.getenv("DEEPSEEK_API_KEY"), 
            base_url=os.getenv("DEEPSEEK_BASE_URL")
        )
        self.model_name = "deepseek-chat" # 或者 "deepseek-ai/DeepSeek-V2.5"
        self.system_prompt = """
        你是一个专业的 AI 算法面试助手。
        你的任务是帮助用户准备 AI 面试，或者回答关于深度学习的技术问题。
        
        1. 对于日常问候（如“你好”、“你是谁”），请直接热情回复，**不要**调用工具。
        2. 对于具体的技术问题（如“什么是 RAG”、“解释下 BERT”），请务必调用 search_knowledge_base 工具检索信息。
        3. 回答要简洁、专业。
        """
        # 对话记忆 (History)
        self.messages = [{"role": "system", "content": self.system_prompt}]

    def chat(self, user_input: str):
        # 1. 添加用户消息
        self.messages.append({"role": "user", "content": user_input})
        
        # 2. 第一轮调用 LLM (思考)
        print("🤖 Agent is thinking...")
        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=self.messages,
            tools=TOOLS_SCHEMA,
            tool_choice="auto" 
        )
        
        response_msg = response.choices[0].message
        tool_calls = response_msg.tool_calls
        
        # 3. 判断是否需要行动
        if tool_calls:
            print(f"🛠️ Agent decided to use tool: {tool_calls[0].function.name}")
            
            # 将 LLM 的“想调用工具”的想法加入历史
            self.messages.append(response_msg)
            
            # 执行所有工具调用
            for tool_call in tool_calls:
                fn_name = tool_call.function.name
                fn_to_call = AVAILABLE_FUNCTIONS[fn_name]
                fn_args = json.loads(tool_call.function.arguments)
                
                # 真正执行函数
                tool_output = fn_to_call(**fn_args)
                
                # 将结果回填给 LLM
                self.messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": fn_name,
                    "content": tool_output
                })
            
            # 4. 第二轮调用 LLM (生成最终回答)
            final_response = self.client.chat.completions.create(
                model=self.model_name,
                messages=self.messages
            )
            reply = final_response.choices[0].message.content
        else:
            # 不需要工具，直接回复
            print("💬 Agent decided to chat directly.")
            reply = response_msg.content
        
        # 将最终回答加入历史
        self.messages.append({"role": "assistant", "content": reply})
        return reply

# --- 测试代码 ---
if __name__ == "__main__":
    agent = RAGAgent()
    
    print("\n--- Test 1: Chit-chat ---")
    print("User: 你好")
    print("Agent:", agent.chat("你好"))
    
    print("\n--- Test 2: Technical Query ---")
    print("User: 讲讲 transformer 是什么")
    print("Agent:", agent.chat("讲讲 transformer 是什么"))

```

## 3. 算法攻坚：验证二叉搜索树 (LC 98)

这是 BST (Binary Search Tree) 系列最经典的题目。

### 题目分析

* **误区**: 很多人只检查 `Left.Val < Root.Val` 和 `Right.Val > Root.Val`。
* **错误案例**:
```text
    5
   / \
  4   6
     / \
    3   7

```


这里 `3 < 6` 且 `7 > 6`，看似局部满足。但 `3` 在 `5` 的右子树里，却比 `5` 小！这是非法的。


* **第一性原理**: 每个节点都有一个**合法的取值区间** `(min_val, max_val)`。
* 根节点: `(-inf, +inf)`
* 左孩子: 必须比爸爸小，所以区间变为 `(parent_min, parent_val)`
* 右孩子: 必须比爸爸大，所以区间变为 `(parent_val, parent_max)`



### 代码实现 (Python)

```python
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right

class Solution:
    def isValidBST(self, root: Optional[TreeNode]) -> bool:
        # 开启递归，初始区间为无穷大
        return self.validate(root, float('-inf'), float('inf'))
    
    def validate(self, node, min_val, max_val):
        # Base Case: 空节点是合法的 BST
        if not node:
            return True
        
        # 当前节点必须在区间 (min_val, max_val) 之间
        # 注意是严格小于/大于，不能等于
        if not (min_val < node.val < max_val):
            return False
        
        # 递归检查左右子树
        # 往左走：上界变小 (必须小于当前节点)
        # 往右走：下界变大 (必须大于当前节点)
        return (self.validate(node.left, min_val, node.val) and 
                self.validate(node.right, node.val, max_val))

```

## 4. 今日任务总结

1. [ ] **Code**: 创建 `src/llm/tools.py` 和 `src/rag/agent.py`，并成功运行 Main 函数测试。
* *验证点*: 问“你好”时不应该看到 `🔍 正在检索` 的日志；问“Transformer”时应该看到。


2. [ ] **Algorithm**: 理解并 AC LeetCode 98。记住**区间传递**的思想，这在后续“二叉树剪枝”等题目中非常通用。
3. [ ] **Git**: 提交代码。
```bash
git add src/llm/tools.py src/rag/agent.py
git commit -m "feat: add simple agent loop and mock search tool"

```



💡 **导师提示**:
如果运行 `src/rag/agent.py` 成功了，你的 AI 面试助手雏形就诞生了！明天我们将把代码“收拾干净”，进行模块化重构，让它看起来像一个成熟的 GitHub 项目。加油！